# TRANSFER LEARNING FOR IMAGE CLASSIFICATION OF COSTA RICAN DISHES USING RESNET50

## Setup for Image Classification with Transfer Learning using ResNet50

In [ ]:
import torch
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader
from torchvision.models import resnet50, ResNet50_Weights
from PIL import Image
import os
import matplotlib.pyplot as plt

# transform for the train data
# Define transformations for training data:
# 1. RandomResizedCrop: Randomly crops and resizes images to 224x224 for consistency and data augmentation.
# 2. RandomHorizontalFlip: Horizontally flips images randomly for augmentation.
# 3. RandomRotation: Randomly rotates the images in a 15 degrees angle.
# 4. ColorJitter: Adjusts brightness and contrast for variety in image appearance.
# 5. ToTensor: Converts images to PyTorch tensors, the format required for model input.
# 6. Normalize: Normalizes images to have a specific mean and standard deviation, aligning with pre-trained model expectations.
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(degrees=25),
    transforms.ColorJitter(brightness=0.5, contrast=0.5),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


# transform for the validation data
# Define transformations for validation data:
# 1. Resize: Increases the size of the image to 256x256 pixels.
# 2. CenterCrop: Crops the center part of the image to 224x224, ensuring it's the same size as the training images.
# 3. ToTensor: Converts the image to a PyTorch tensor, suitable for model input.
# 4. Normalize: Normalizes the image with the specified mean and standard deviation, matching the training data's normalization.
validation_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# load the datasets
path_traning_data = "../dataset/trainig-data"
path_validation_data = "../dataset/test-data"
train_dataset = ImageFolder(root=path_traning_data, transform=train_transforms)
validation_dataset = ImageFolder(root=path_validation_data, transform=validation_transforms)

# define the data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=32, shuffle=False)

# load the pre-trained ResNet50 model
model = resnet50(weights=ResNet50_Weights.IMAGENET1K_V1)

# Get the number of input features for the final fully connected layer of the pre-trained model
num_ftrs = model.fc.in_features

# Replace the final fully connected layer with a new one tailored to the number of classes in the dataset
# This is necessary to adapt the pre-trained model for the specific classification task
model.fc = torch.nn.Linear(num_ftrs, len(train_dataset.classes))

# move the model to a GPU if it is available
# Check and set the device for Apple M1 (MPS)
#device = torch.device("cpu")
device = torch.device("mps") if torch.backends.mps.is_available() else torch.device("cpu")
model = model.to(device)

# define loss function and optimizer
#criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# Define loss function and optimizer with increased weight decay for L2 regularization
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)  # Added weight decay

# set numbers of epochs
num_epochs= 13